Find the info needed to run specfem for each event in catalogue:
* average strike
* average dip
* average rake
* fault coordinates
* mo

In [1]:
# import modules
import math
import numpy as np
import pandas as pd

from rsqsim_api.catalogue.catalogue import RsqSimCatalogue
from rsqsim_api.fault.multifault import RsqSimMultiFault
import os
from shapely.geometry.polygon import Polygon
import geopandas as gpd

Read in catalogue and fault model

In [2]:
script_dir = os.path.abspath('')
fault_dir = "../../../data/shaw2021/rundir5091"
catalogue_dir = os.path.join(fault_dir,"specfem")

fault_model = RsqSimMultiFault.read_fault_file_bruce(os.path.join(script_dir, fault_dir, "zfault_Deepen.in"),
                                                     os.path.join(script_dir, fault_dir, "znames_Deepen.in"),
                                                     transform_from_utm=True)
whole_catalogue = RsqSimCatalogue.from_csv_and_arrays(os.path.join(catalogue_dir, "single_fault_200yr"))

In [40]:
#get specific event of interest: 448545
ev_number=448545
ev_cat=whole_catalogue.filter_by_events(event_number=ev_number)
EOI=ev_cat.first_event(fault_model)
EOI_fault=EOI.faults[0]

In [4]:
#write out parameters of interest
out_name=f'event_{ev_number}.txt'
with open(os.path.join(script_dir,out_name),'w') as f_out:
    f_out.write(f'#Evid mean_strike mean_dip mean_rake mean_slip(m) Mo\n')
    f_out.write(f'{ev_number} {EOI.mean_strike:.1f} {EOI.mean_dip:.1f} {EOI.mean_rake:.1f} {EOI.mean_slip:.1f} {EOI.m0:.2e}')

In [36]:
#write out trace
EOItrace=gpd.GeoSeries(EOI_fault.trace,crs="EPSG:2193")
#convert to lat/ lon
EOItrace_ll=EOItrace.to_crs("EPSG:4326")


In [99]:
EOI_outline=gpd.GeoSeries(EOI_fault.fault_outline,crs="EPSG:2193")
EOI_outline_ll=EOI_outline.to_crs("EPSG:4326").iloc[0]
as_xyz=np.reshape(EOI_outline_ll.exterior.coords[:],(len(EOI_outline_ll.exterior.coords[:]),3))
EOI_pandas=pd.DataFrame(as_xyz)
EOI_pandas.to_csv(f'event_{ev_number}_fault_outline.csv',header=False,index=False)

In [7]:
EOI.slip_dist_to_txt(os.path.join(script_dir,f'event_{ev_number}_slip.txt'),nztm_to_lonlat=True)

In [11]:
whole_catalogue.event_list

array([447345, 447345, 447346, ..., 448853, 448853, 448853], dtype=int32)